# Sawyer Jacobson
## Data Science Capstone, Midterm Project

Planned things to do:

- Data Preprocessing
    - Fill in missing values
    - Clean the 'target' salary data
    - Split into train and test data
- EDA
    - Plot distributions of variables
        - Games played is pretty big
    - Check for high correlations
    - Potentially look for outliers
    - Separate by position
- Modeling
    - Unsupervised clustering as feature engineering
        - Kmeans separately on forwards and defensemen
        - Used to potentially categorize offensive and defensive players in each group since that is where the clusters will most likely align
    - Potential feature selection since there are a fair amount of features for the number of observations
    - Supervised learning: predicting salary
        - Use and optimize a few different models with RMSE as the error metric


In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import plotnine
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import scipy
from sklearn import metrics
import sklearn as sk
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data Preprocessing

Here we will read in the data and explore, fill in missing values, and drop redundant variables as needed. The salary dataset will also be cleaned and merged with the player statistics dataset to determine how many observations will be present.

Additionally, we must check for duplicate players on different teams throughout each season to account for trades. These cases will need to be combined to get the players complete statistics for the season. This should be as simple as aggregating numeric variables and leaving others along.

In [2]:
player_df = pd.read_csv("data/player_season.csv")
salary_full = pd.read_csv("data/player_salary.csv")
salary_full.rename(columns = {"player": "fullName"}, inplace = True)
# updating the max seen columns to help see the data
pd.set_option("max_columns", player_df.shape[1])

player_df.head()
salary_full.head()

player_df.shape
salary_full.shape

player_df.isnull().sum()

,position,team,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,rookie,shootsCatches,season_type,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,rosterStatus,goalsInOvertime,goalsTrailingByThreePlus
0,C,New Jersey Devils,8470619,Brian Boyle,/api/v1/people/8470619,Brian,Boyle,9.0,1984-12-18,Hingham,MA,USA,USA,"6' 6""",245,False,False,L,R,20182019,980:48,6,18,38,123,73,145,7,9,192:05,676:00,38,51.05,14.63,4,0,1,2,112:43,50,-14,24,1255,13:26,09:15,01:32,02:37,4.0,6.0,8.0,0.0,1.0,2.0,4.0,3.0,7.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,R,New Jersey Devils,8471226,Drew Stafford,/api/v1/people/8471226,Drew,Stafford,18.0,1985-10-30,Milwaukee,WI,USA,USA,"6' 2""",215,False,False,R,R,20182019,699:08,8,5,18,77,57,50,1,2,50:26,646:27,18,41.02,6.49,0,0,0,0,02:15,19,-7,13,868,12:15,11:20,00:02,00:53,2.0,3.0,NaN,0.0,2.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,C,New Jersey Devils,8471233,Travis Zajac,/api/v1/people/8471233,Travis,Zajac,14.0,1985-05-13,Winnipeg,MB,CAN,CAN,"6' 2""",185,True,False,R,R,20182019,1541:40,27,19,20,120,80,66,5,12,211:31,1112:40,20,58.16,15.83,3,1,2,2,217:29,38,-25,46,1818,19:16,13:54,02:43,02:38,7.0,8.0,3.0,0.0,0.0,1.0,2.0,1.0,6.0,4.0,2.0,1.0,0.0,0.0,35.0,False,False,Y,1.0,3.0
3,D,New Jersey Devils,8472382,Andy Greene,/api/v1/people/8472382,Andy,Greene,4.0,1982-10-30,Trenton,MI,USA,USA,"5' 11""",190,True,False,L,R,20182019,1708:22,20,5,16,104,82,67,0,2,09:09,1363:18,16,0.00,4.81,0,0,0,4,335:55,208,-9,25,2027,20:50,16:37,04:05,00:06,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,38.0,False,False,Y,NaN,2.0
4,D,New Jersey Devils,8473468,Eric Gryba,/api/v1/people/8473468,Eric,Gryba,2.0,1988-04-14,Saskatoon,SK,CAN,CAN,"6' 4""",222,False,False,R,R,20182019,152:57,0,0,10,11,10,21,0,0,00:07,140:09,10,0.00,0.00,0,0,0,0,12:41,16,-1,0,194,15:17,14:00,01:16,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,fullName,position,age,base_salary,signing_bonus,perf_bonus,total_salary,total_cap_hit,adjusted_cap_hit,cap_pct,team,season
0,Aaron Dell,G,29,"$1,950,000","$250,000",-,"$2,200,000","$1,900,000","$1,900,000",2.37,san-jose-sharks,2018/2019
1,Aaron Dell,G,30,"$1,600,000",-,-,"$1,600,000","$1,900,000","$1,900,000",2.62,san-jose-sharks,2019/2020
2,Aaron Ekblad,D,22,"$4,000,000",-,-,"$7,000,000","$7,500,000","$7,500,000",9.76,florida-panthers,2018/2019
3,Aaron Ekblad,D,23,"$5,000,000","$4,000,000",-,"$9,000,000","$7,500,000","$7,500,000",9.38,florida-panthers,2019/2020
4,Adam Boqvist,D,19,"$832,500","$92,500","$850,000","$705,712","$894,167","$658,559",0.95,chicago-blackhawks,2019/2020


(1947, 67)

(1482, 12)

position                       0
team                           0
id                             0
fullName                       0
link                           0
                            ... 
alternateCaptain             242
captain                      242
rosterStatus                 242
goalsInOvertime             1677
goalsTrailingByThreePlus    1371
Length: 67, dtype: int64

In [3]:
# some player names are longer or have slightly different capitalization so I'm
# going to hand fix as many as I can
salary_full.loc[salary_full.fullName == "Cameron Atkinson", ['fullName']] = "Cam Atkinson"
salary_full.loc[salary_full.fullName == "Alexander Debrincat", ['fullName']] = "Alex DeBrincat"
salary_full.loc[salary_full.fullName == "Alexander Killorn", ['fullName']] = "Alex Killorn"
salary_full.loc[salary_full.fullName == "Charles Mcavoy", ['fullName']] = "Charlie McAvoy"
salary_full.loc[salary_full.fullName == "Chris Tanev", ['fullName']] = "Christopher Tanev"
salary_full.loc[salary_full.fullName == "Dylan Demelo", ['fullName']] = "Dylan DeMelo"

salary_full.loc[salary_full.fullName == "Evgeni Dadonov", ['fullName']] = "Evgenii Dadonov"
salary_full.loc[salary_full.fullName == "Gabe Vilardi", ['fullName']] = "Gabriel Vilardi"
salary_full.loc[salary_full.fullName == "Gaëtan Haas", ['fullName']] = "Gaetan Haas"
salary_full.loc[salary_full.fullName == "J.T Compher", ['fullName']] = "J.T. Compher"
salary_full.loc[salary_full.fullName == "J.T. Brown", ['fullName']] = "JT Brown"
salary_full.loc[salary_full.fullName == "Jacob De La Rose", ['fullName']] = "Jacob de la Rose"
salary_full.loc[salary_full.fullName == "Jacob Muzzin", ['fullName']] = "Jake Muzzin"
salary_full.loc[salary_full.fullName == "James Van Riemsdyk", ['fullName']] = "James van Riemsdyk"

salary_full.loc[salary_full.fullName == "Joshua Brown", ['fullName']] = "Josh Brown"
salary_full.loc[salary_full.fullName == "Joshua Morrissey", ['fullName']] = "Josh Morrissey"
salary_full.loc[salary_full.fullName == "Kenneth Agostino", ['fullName']] = "Kenny Agostino"
salary_full.loc[salary_full.fullName == "Kurtis Macdermid", ['fullName']] = "Kurtis MacDermid"
salary_full.loc[salary_full.fullName == "MacKenzie MacEachern", ['fullName']] = "Mackenzie MacEachern"
salary_full.loc[salary_full.fullName == "Magnus Paajarvi-Svensson", ['fullName']] = "Magnus Paajarvi"
salary_full.loc[salary_full.fullName == "Mats Zuccarello-Aasen", ['fullName']] = "Mats Zuccarello"
salary_full.loc[salary_full.fullName == "Matthew Benning", ['fullName']] = "Matt Benning"
salary_full.loc[salary_full.fullName == "Matthew Grzelcyk", ['fullName']] = "Matt Grzelcyk"
salary_full.loc[salary_full.fullName == "Michael Matheson", ['fullName']] = "Mike Matheson"

salary_full.loc[salary_full.fullName == "Michael Mcleod", ['fullName']] = "Michael McLeod"
salary_full.loc[salary_full.fullName == "Nicholas Paul", ['fullName']] = "Nick Paul"
salary_full.loc[salary_full.fullName == "Nicholas Ritchie", ['fullName']] = "Nick Ritchie"
salary_full.loc[salary_full.fullName == "Nick Shore", ['fullName']] = "Nicholas Shore"
salary_full.loc[salary_full.fullName == "Nicolas Petan", ['fullName']] = "Nic Petan"
salary_full.loc[salary_full.fullName == "Nikolay Prokhorkin", ['fullName']] = "Nikolai Prokhorkin"
salary_full.loc[salary_full.fullName == "Patrick Maroon", ['fullName']] = "Pat Maroon"
salary_full.loc[salary_full.fullName == "Paul Ladue", ['fullName']] = "Paul LaDue"
salary_full.loc[salary_full.fullName == "Philip Kessel", ['fullName']] = "Phil Kessel"
salary_full.loc[salary_full.fullName == "Phillip di Giuseppe", ['fullName']] = "Phillip Di Giuseppe"

salary_full.loc[salary_full.fullName == "Samuel Blais", ['fullName']] = "Sammy Blais"
salary_full.loc[salary_full.fullName == "T.J. Brodie", ['fullName']] = "TJ Brodie"
salary_full.loc[salary_full.fullName == "Theodor Blueger", ['fullName']] = "Teddy Blueger"
salary_full.loc[salary_full.fullName == "Thomas Wilson", ['fullName']] = "Tom Wilson"
salary_full.loc[salary_full.fullName == "Trevor Van Riemsdyk", ['fullName']] = "Trevor van Riemsdyk"
salary_full.loc[salary_full.fullName == "Vincent Hinostroza", ['fullName']] = "Vinnie Hinostroza"
salary_full.loc[salary_full.fullName == "Zachary Werenski", ['fullName']] = "Zach Werenski"

In [4]:
# removing unneeded columns and cleaning the salary columns
def clean_money(x):
    return x.str.replace("$", '').str.replace(",", '').str.replace("-", "0")
money_cols = ['base_salary', 'signing_bonus', 'perf_bonus', 'total_salary', 'total_cap_hit']

salary_full[money_cols] = salary_full[money_cols].apply(clean_money).astype(int)
salary_df = (salary_full.
             query("total_cap_hit != '-'").
             assign(
                 total_salary = lambda x: x.base_salary + x.signing_bonus + x.perf_bonus,
                 season = lambda x: x.season.str.replace("/", '')
             ).
             filter(['fullName', 'total_salary', 'total_cap_hit', 'season'])
            )
# salary_df.isnull().sum()

salary_df.head()
salary_df.describe()

<ipython-input-4-9b4312ce2a2a>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


,fullName,total_salary,total_cap_hit,season
0,Aaron Dell,2200000,1900000,20182019
1,Aaron Dell,1600000,1900000,20192020
2,Aaron Ekblad,4000000,7500000,20182019
3,Aaron Ekblad,9000000,7500000,20192020
4,Adam Boqvist,1775000,894167,20192020


,total_salary,total_cap_hit
count,1.482000e+03,1.482000e+03
mean,3.226768e+06,2.975673e+06
std,2.696134e+06,2.424949e+06
min,6.500000e+05,0.000000e+00
25%,9.250000e+05,8.941660e+05
50%,2.337500e+06,2.100000e+06
75%,5.000000e+06,4.833333e+06
max,1.600000e+07,1.250000e+07


In [5]:
pd.set_option("max_info_columns", player_df.shape[1])
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 67 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   position                     1947 non-null   object 
 1   team                         1947 non-null   object 
 2   id                           1947 non-null   int64  
 3   fullName                     1947 non-null   object 
 4   link                         1947 non-null   object 
 5   firstName                    1947 non-null   object 
 6   lastName                     1947 non-null   object 
 7   primaryNumber                1944 non-null   float64
 8   birthDate                    1947 non-null   object 
 9   birthCity                    1947 non-null   object 
 10  birthStateProvince           1393 non-null   object 
 11  birthCountry                 1947 non-null   object 
 12  nationality                  1947 non-null   object 
 13  height            

In [6]:
# player_df.groupby(by = ['fullName', 'season']).sum()
player_df.head()

,position,team,id,fullName,link,firstName,lastName,primaryNumber,birthDate,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,rookie,shootsCatches,season_type,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,rosterStatus,goalsInOvertime,goalsTrailingByThreePlus
0,C,New Jersey Devils,8470619,Brian Boyle,/api/v1/people/8470619,Brian,Boyle,9.0,1984-12-18,Hingham,MA,USA,USA,"6' 6""",245,False,False,L,R,20182019,980:48,6,18,38,123,73,145,7,9,192:05,676:00,38,51.05,14.63,4,0,1,2,112:43,50,-14,24,1255,13:26,09:15,01:32,02:37,4.0,6.0,8.0,0.0,1.0,2.0,4.0,3.0,7.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,R,New Jersey Devils,8471226,Drew Stafford,/api/v1/people/8471226,Drew,Stafford,18.0,1985-10-30,Milwaukee,WI,USA,USA,"6' 2""",215,False,False,R,R,20182019,699:08,8,5,18,77,57,50,1,2,50:26,646:27,18,41.02,6.49,0,0,0,0,02:15,19,-7,13,868,12:15,11:20,00:02,00:53,2.0,3.0,NaN,0.0,2.0,6.0,2.0,2.0,1.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,C,New Jersey Devils,8471233,Travis Zajac,/api/v1/people/8471233,Travis,Zajac,14.0,1985-05-13,Winnipeg,MB,CAN,CAN,"6' 2""",185,True,False,R,R,20182019,1541:40,27,19,20,120,80,66,5,12,211:31,1112:40,20,58.16,15.83,3,1,2,2,217:29,38,-25,46,1818,19:16,13:54,02:43,02:38,7.0,8.0,3.0,0.0,0.0,1.0,2.0,1.0,6.0,4.0,2.0,1.0,0.0,0.0,35.0,False,False,Y,1.0,3.0
3,D,New Jersey Devils,8472382,Andy Greene,/api/v1/people/8472382,Andy,Greene,4.0,1982-10-30,Trenton,MI,USA,USA,"5' 11""",190,True,False,L,R,20182019,1708:22,20,5,16,104,82,67,0,2,09:09,1363:18,16,0.00,4.81,0,0,0,4,335:55,208,-9,25,2027,20:50,16:37,04:05,00:06,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,38.0,False,False,Y,NaN,2.0
4,D,New Jersey Devils,8473468,Eric Gryba,/api/v1/people/8473468,Eric,Gryba,2.0,1988-04-14,Saskatoon,SK,CAN,CAN,"6' 4""",222,False,False,R,R,20182019,152:57,0,0,10,11,10,21,0,0,00:07,140:09,10,0.00,0.00,0,0,0,0,12:41,16,-1,0,194,15:17,14:00,01:16,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# converting the time on ice columns to just minutes, removing the seconds after the : since
# those can be considered negligible in most cases

# function to clean the time on ice from minutes:seconds format to a decimal
def clean_toi(x):
    time = x.str.split(":")
    minutes = time.str.get(0)
    seconds = time.str.get(1).astype(int)*100/60
    return minutes + "." + seconds.round().astype(int).astype(str)

time_on_ice_cols = ['timeOnIce', 'powerPlayTimeOnIce', 'evenTimeOnIce', 'shortHandedTimeOnIce', 
                    'timeOnIcePerGame', 'evenTimeOnIcePerGame', 'shortHandedTimeOnIcePerGame', 
                    'powerPlayTimeOnIcePerGame']
# this isn't working quite right...
player_df[time_on_ice_cols] = player_df[time_on_ice_cols].apply(clean_toi).astype(float)

In [8]:
player_df['season'] = player_df['season'].astype(str)

# converting height to inches
r = re.compile(r"([0-9]+)' ([0-9]*\.?[0-9]+)\"")
def get_inches(height):
    m = r.match(height)
    if m == None:
        return float('NaN')
    else:
        return int(m.group(1))*12 + float(m.group(2))
    
player_df['height'] = player_df['height'].apply(get_inches)

In [9]:
# dropping unnecessary columns
player_final = player_df.drop(columns = ['team', "primaryNumber", 'birthDate', 'id', 'link', 'firstName', "lastName", 
                         'birthCity', 'birthStateProvince', 'birthCountry',
                         'active', 'rosterStatus', 'season_type']).copy()
# making the rookie column an integer
player_final["rookie"] = player_final.rookie.astype(int)

In [10]:
player_final.query('fullName == "Par Lindholm"')

,position,fullName,nationality,height,weight,rookie,shootsCatches,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,goalsInOvertime,goalsTrailingByThreePlus
326,C,Par Lindholm,SWE,72.0,181,0,L,20182019,737.53,12,1,18,65,65,58,0,0,4.50,629.87,18,50.12,1.54,0,0,0,0,103.17,55,5,13,1136,11.33,9.68,1.58,0.7,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,C,Par Lindholm,SWE,72.0,181,0,L,20182019,737.53,12,1,18,65,65,58,0,0,4.50,629.87,18,50.12,1.54,0,0,0,0,103.17,55,5,13,1136,11.33,9.68,1.58,0.7,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1181,C,Par Lindholm,SWE,72.0,181,0,L,20192020,449.78,3,3,4,35,40,23,0,0,0.17,380.98,4,45.83,8.60,1,0,0,0,68.63,26,6,6,632,11.23,9.52,1.70,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# columns for one-hot-encoding
one_hot_columns = ["position", 'shootsCatches']

# changing missing values to false and making integers for the captain columns
player_final['alternateCaptain'] = player_final.alternateCaptain.fillna("False").astype(bool).astype(int)
player_final['captain'] = player_final.captain.fillna("False").astype(bool).astype(int)

,position,fullName,nationality,shootsCatches,season,alternateCaptain,captain
0,C,Brian Boyle,USA,L,20182019,NaN,NaN
1,R,Drew Stafford,USA,R,20182019,NaN,NaN
2,C,Travis Zajac,CAN,R,20182019,False,False
3,D,Andy Greene,USA,L,20182019,False,False
4,D,Eric Gryba,CAN,R,20182019,NaN,NaN
...,...,...,...,...,...,...,...
1942,C,Nicolas Roy,CAN,R,20192020,False,False
1943,C,Gage Quinney,USA,L,20192020,False,False
1944,D,Nicolas Hague,CAN,L,20192020,False,False
1945,C,Cody Glass,CAN,R,20192020,False,False


In [19]:
# checking which observations are duplicates
player_final_key = player_final.groupby(by = ['fullName', 'season']).cumcount() == 0
# filtering out the duplicates
player_clean = player_final[player_final_key].reset_index(drop=True).fillna(0)
# need to get matching data for the object columns to merge back in
# straight forward since they are values that won't change in a season
player_clean

,position,fullName,nationality,shootsCatches,season
0,C,Brian Boyle,USA,L,20182019
1,R,Drew Stafford,USA,R,20182019
2,C,Travis Zajac,CAN,R,20182019
3,D,Andy Greene,USA,L,20182019
4,D,Eric Gryba,CAN,R,20182019
...,...,...,...,...,...
1942,C,Nicolas Roy,CAN,R,20192020
1943,C,Gage Quinney,USA,L,20192020
1944,D,Nicolas Hague,CAN,L,20192020
1945,C,Cody Glass,CAN,R,20192020


,position,fullName,nationality,height,weight,rookie,shootsCatches,season,timeOnIce,assists,goals,pim,shots,games,hits,powerPlayGoals,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shotPct,gameWinningGoals,overTimeGoals,shortHandedGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,plusMinus,points,shifts,timeOnIcePerGame,evenTimeOnIcePerGame,shortHandedTimeOnIcePerGame,powerPlayTimeOnIcePerGame,goalsInFirstPeriod,goalsInSecondPeriod,goalsInThirdPeriod,emptyNetGoals,shootOutGoals,shootOutShots,goalsTrailingByOne,goalsTrailingByTwo,goalsWhenTied,goalsLeadingByOne,goalsLeadingByTwo,goalsLeadingByThreePlus,penaltyGoals,penaltyShots,currentAge,alternateCaptain,captain,goalsInOvertime,goalsTrailingByThreePlus
0,C,Brian Boyle,USA,78.0,245,0,L,20182019,980.80,6,18,38,123,73,145,7,9,192.80,676.00,38,51.05,14.63,4,0,1,2,112.72,50,-14,24,1255,13.43,9.25,1.53,2.62,4.0,6.0,8.0,0.0,1.0,2.0,4.0,3.0,7.0,1.0,2.0,1.0,0.0,0.0,0.0,1,1,0.0,0.0
1,R,Drew Stafford,USA,74.0,215,0,R,20182019,699.13,8,5,18,77,57,50,1,2,50.43,646.45,18,41.02,6.49,0,0,0,0,2.25,19,-7,13,868,12.25,11.33,0.30,0.88,2.0,3.0,0.0,0.0,2.0,6.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0
2,C,Travis Zajac,CAN,74.0,185,0,R,20182019,1541.67,27,19,20,120,80,66,5,12,211.52,1112.67,20,58.16,15.83,3,1,2,2,217.48,38,-25,46,1818,19.27,13.90,2.72,2.63,7.0,8.0,3.0,0.0,0.0,1.0,2.0,1.0,6.0,4.0,2.0,1.0,0.0,0.0,35.0,0,0,1.0,3.0
3,D,Andy Greene,USA,71.0,190,0,L,20182019,1708.37,20,5,16,104,82,67,0,2,9.15,1363.30,16,0.00,4.81,0,0,0,4,335.92,208,-9,25,2027,20.83,16.62,4.80,0.10,1.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,38.0,0,0,0.0,2.0
4,D,Eric Gryba,CAN,76.0,222,0,R,20182019,152.95,0,0,10,11,10,21,0,0,0.12,140.15,10,0.00,0.00,0,0,0,0,12.68,16,-1,0,194,15.28,14.00,1.27,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,C,Nicolas Roy,CAN,76.0,205,0,R,20192020,320.20,5,5,8,42,28,39,1,1,2.72,302.13,8,48.18,11.90,1,0,0,0,15.17,7,3,10,439,11.42,10.78,0.53,0.80,3.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,24.0,0,0,0.0,0.0
1785,C,Gage Quinney,USA,71.0,200,1,L,20192020,29.13,1,0,0,2,3,2,0,0,0.00,29.13,0,47.05,0.00,0,0,0,0,0.00,0,-1,1,42,9.70,9.70,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0,0,0.0,0.0
1786,D,Nicolas Hague,CAN,78.0,230,0,L,20192020,611.40,10,1,32,54,38,69,1,4,34.73,575.72,32,0.00,1.90,0,0,0,0,0.95,26,0,11,802,16.80,15.15,0.20,0.90,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,22.0,0,0,0.0,0.0
1787,C,Cody Glass,CAN,75.0,206,0,R,20192020,521.50,7,5,6,51,39,15,1,6,86.83,433.20,6,41.95,9.80,1,0,0,0,1.20,6,-7,12,627,13.35,11.10,0.20,2.22,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,22.0,0,0,0.0,0.0


In [20]:
player_full = pd.merge(
    salary_df,
    player_clean, 
    on = ['fullName', "season"], 
    how = "inner")

In [21]:
# checking for nulls one last time
# all clear!
test.isnull().sum()

fullName                       0
total_salary                   0
total_cap_hit                  0
season                         0
position                       0
nationality                    0
height                         0
weight                         0
rookie                         0
shootsCatches                  0
timeOnIce                      0
assists                        0
goals                          0
pim                            0
shots                          0
games                          0
hits                           0
powerPlayGoals                 0
powerPlayPoints                0
powerPlayTimeOnIce             0
evenTimeOnIce                  0
penaltyMinutes                 0
faceOffPct                     0
shotPct                        0
gameWinningGoals               0
overTimeGoals                  0
shortHandedGoals               0
shortHandedPoints              0
shortHandedTimeOnIce           0
blocked                        0
plusMinus 

In [ ]:
player